In [ ]:
from scipy import io as spio
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from IPython.display import clear_output
from data import get_training_data, get_test_data
tfe = tf.contrib.eager # Shorthand for some symbols
tf.enable_eager_execution()

## Emnist
see https://arxiv.org/pdf/1702.05373v1.pdf for breakdown of dataset

In [ ]:
# Load balanced letters dataset from csv
df_train = pd.read_csv("data/emnist-letters-train.csv")

# Data Shape
data shape of df_train is:

column 0 is the class

columns 1 to 785 are the image data

In [ ]:
# get labels from first column
df_train_y = df_train.iloc[:,[0]]
# get 1d image data from other 784 columns
df_train_x = df_train.iloc[:, 1:785]

In [ ]:
df_train_y.head()

# Class transform
By default the Y classes are made up of 26 numbers each representing a letter of the alphabet,
as we only care about the characters 'm' and 'f', we will use a piecewise function to set anything that is not 'm' or 'f' to 0,
any class that is 'f' to 1, and any class that is 'm' to 2.

| class | mapping |
|-------|---------|
|   0   | unknown |
|   1   |    f    |
|   2   |    m    |

In [ ]:
y = np.asarray(df_train_y)
train_y = np.piecewise(y, [(y != 6) & (y != 13), y == 6, y == 13], [0, 1, 2])

In [ ]:
df_train_x.head()

# Letter Data Transform
The training examples are 1d (1x784) and need to be resized to 28x28

In [ ]:
train_x = df_train_x.values.reshape((88799, 28, 28))

Then the images need to be rotated 90 degrees, the 0th axis of our train_x is the image index itself, thus we only want to rotate axis 1 and 2 as they are the actual image data

The rot90 function is able to rotate an entire numpy matrix

In [ ]:
train_x = np.fliplr(train_x)
train_x = np.rot90(train_x, axes=(2, 1))

# Plot F's and M's

In [ ]:
figure = plt.figure(figsize=(28, 28))
columns = 4
rows = 2
image_index = 0
for i in range(1, columns * rows + 1):
    figure.add_subplot(rows, columns, i)
    while train_y[image_index] == 0:
        image_index += 1
    print(image_index)
    plt.imshow(train_x[image_index], cmap='gray')
    image_index +=1

# Creating the Tensorflow model
The model is created similiar to the example given at https://www.tensorflow.org/guide/eager

The network is made up of several convolutional layers with maxpool layers inbetween

In [ ]:
class MOFCnn(tf.keras.Model):
    def __init__(self):
        super(MOFCnn, self).__init__()
        # CNN layers
        self.cnn1 = tf.keras.layers.Conv2D(32, 3, input_shape=(28, 28, 1), activation='relu')
        self.cnn2 = tf.keras.layers.Conv2D(64, 3, activation='relu')
        self.cnn3 = tf.keras.layers.Conv2D(128, 3, activation='relu')
        
        # maxpool layers:
        self.maxpool = tf.layers.MaxPooling2D((2, 2), (2,2))
        
        # flatten layer:
        self.flatten = tf.layers.Flatten()
        
        # fully connected layers
        self.dense1 = tf.layers.Dense(100, activation='relu')
        self.denseOutput = tf.layers.Dense(3, activation='softmax')
        
        # dropout
        self.dropoutFull = tf.layers.Dropout(0.5)
        self.dropoutHalf = tf.layers.Dropout(0.25)
    
    def call(self, input):
        result = self.cnn1(input)
        result = self.maxpool(result)
        result = self.cnn2(result)
        result = self.maxpool(result)
        result = self.cnn3(result)
        result = self.maxpool(result)
        result = self.flatten(result)
        result = self.dense1(result)
        result = self.dropoutFull(result)
        result = self.denseOutput(result)
        return result
    
    @staticmethod
    def loss(model, x, y):
        prediction = model(x)
        return tf.losses.softmax_cross_entropy(onehot_labels=y, logits=prediction)
    
    @staticmethod
    def grad(model, inputs, targets):
        with tf.GradientTape() as tape:
            loss_value = MOFCnn.loss(model, inputs, targets)
        return tape.gradient(loss_value, model.variables)
    
    @staticmethod
    def accuracy(predictions, labels):
        model_pred = tf.argmax(predictions, axis=1,output_type=tf.int64)
        actual_labels = tf.argmax(labels, axis=1, output_type=tf.int64)
        return tf.reduce_sum(tf.cast(tf.equal(model_pred, actual_labels),dtype=tf.float32)) / float(predictions.shape[0].value)

In [ ]:
def trainMofCnn(model, x, y, batch_size, number_of_epochs):
    optimizer = tf.train.AdamOptimizer()
    x = tf.data.Dataset.from_tensor_slices(x)
    y = tf.data.Dataset.from_tensor_slices(y)
    data = tf.data.Dataset.zip((x, y)).batch(batch_size)
    for _ in range(number_of_epochs):
        for xs, ys in data:
            clear_output(True)
            grads = MOFCnn.grad(model, xs, ys)
            optimizer.apply_gradients(zip(grads, model.variables))
            loss = MOFCnn.loss(model, xs, ys)
            predictions = model(xs)
            print("loss: {:.3f}".format(loss))
            print(ys)
            #print(tf.contrib.metrics.accuracy(predictions, ys))
        
        

# Get test and training data

In [ ]:
train_x, train_y = get_training_data()

In [ ]:
test_x, test_y = get_test_data()

# Train Model

In [ ]:
model = MOFCnn()

In [ ]:
trainMofCnn(model, train_x, train_y, 512, 1)

In [ ]:
tf.argmax(model(train_x[0:50]), axis=0)

In [ ]:
train_y[5]

In [ ]:
model.variables

In [ ]:
model([train_x[3]])